<h1 style="text-align:center"> <b>Image Classification using Transfer Learning in PyTorch <b/><h1>

Face masks have become a common public sight in the last few months. The Centers for Disease Control (CDC) recently advised the use of simple cloth face coverings to slow the spread of the virus and help people who may have the virus and do not know it from transmitting it to others. Wearing masks is broadly recognised as critical to reducing community transmission and limiting touching of the face.

In a time of concerns about slowing the transmission of COVID-19, increased surveillance combined with AI solutions can improve monitoring and reduce the human effort needed to limit the spread of this disease. The objective of this challenge is to create an image classification machine learning model to accurately predict the likelihood that an image contains a person wearing a face mask, or not. The total dataset contains 1,800+ images of people either wearing masks or not.

Our machine learning solution will help policymakers, law enforcement, hospitals, and even commercial businesses ensure that masks are being worn appropriately in public. These solutions can help in the battle to reduce community transmission of COVID-19

<h2>Import librarys and modules</h2>

In [0]:
import pandas as pd
import numpy as np
import torchvision
import os
from torchvision import transforms, utils
from PIL import Image
from torchvision.transforms import ToTensor
from torch.autograd import Variable
import torch
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib.pyplot as plt
import PIL 
from torch import nn
import time
import random
from matplotlib import gridspec

In [0]:
print(torchvision.__version__)
print(torch.__version__)
print(PIL.__version__)

0.5.0
1.4.0
7.0.0


<h2>Set a seed for the random initialization of weights provided by the nn module</h2>

In [0]:
manualSeed = 1
random_seed= 42
np.random.seed(manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)
# if you are suing GPU
torch.cuda.manual_seed(manualSeed)
torch.cuda.manual_seed_all(manualSeed)


torch.backends.cudnn.enabled = False 
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

<h2>Download Data</h2>

In [0]:
!wget -O images.zip --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1vtO5_FB--urbF09XYoxbSAEQW-yqwoi6' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1vtO5_FB--urbF09XYoxbSAEQW-yqwoi6" && rm -rf /tmp/cookies.txt

In [0]:
!wget -O train.csv --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1XQ-RfUG1C_6LUZdRUe9nAU77I_AZFH-F' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1XQ-RfUG1C_6LUZdRUe9nAU77I_AZFH-F" && rm -rf /tmp/cookies.txt

In [0]:
!unzip '/content/images.zip'

<h2>Split Images using Train.csv  provided by Zindi to test and train</h2>

In [0]:
train = pd.read_csv('/content/train.csv')

train_images_list = train['image'].tolist()

images_list = os.listdir('/content/IMG/images')

test_images_list = [fn for fn in images_list if fn not in train_images_list]


<h4>Applying Transforms to the Data,
we just resize the images to 256×256 and crop out the center 224×224 in order to be able to use them with the pretrained model. Then the image is transformed into a tensor and normalized by the mean and standard deviation of all images in ImageNet</h4>

In [0]:
image_transform=transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])

<h2>Create Test images DateFrame</h2>

In [0]:
sub=pd.DataFrame({
    'image':test_images_list,
     'target':0
})


<h2>Download Pre-trained  model</h2>





In [18]:
!git clone 'https://github.com/Bouguedra-Adem/Spot-the-Mask-Challenge-by_ZindiWeekendz'

Cloning into 'Spot-the-Mask-Challenge-by_ZindiWeekendz'...
remote: Enumerating objects: 6, done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 6
Unpacking objects: 100% (6/6), done.


In [35]:
resnet50 = torchvision.models.resnet50(pretrained=True)

for param in resnet50.parameters():
    param.requires_grad = True

fc_inputs = resnet50.fc.in_features
 
resnet50.fc = nn.Sequential(
    nn.Linear(fc_inputs, 128),
    nn.ReLU(inplace=True),
    nn.Dropout(0.5),
    nn.Linear(128,2),
    nn.LogSoftmax(dim=1) # For using NLLLoss()
)
model = resnet50.to('cuda:0') 


model.load_state_dict(torch.load('/content/Spot-the-Mask-Challenge-by_ZindiWeekendz/Pre_trained_model.pt/model.pt'))
Resnet50_Bouguedra_Adem=model.eval()
Resnet50_Bouguedra_Adem



ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [0]:
device = torch.device("cuda:0")

<h2>Test<h2>

In [0]:
test_img=[]
for img in sub['image'] :
          img_path="/content/IMG/images/"+str(img)
          test_image = Image.open(img_path).convert('RGB')
        
          test_image_tensor = image_transform(test_image)
          if torch.cuda.is_available():
                test_image_tensor = test_image_tensor.view(1, 3, 224, 224).cuda()
          else:
                test_image_tensor = test_image_tensor.view(1, 3, 224, 224)

          #test_image_tensor = test_image_tensor.view(1, 3, 224, 224)
          with torch.no_grad():
            Resnet50_Bouguedra_Adem.eval()
            out=Resnet50_Bouguedra_Adem(test_image_tensor)
            ps = torch.exp(out)
            topk, topclass = ps.topk(1, dim=1,)
            sub.loc[sub['image']==img,'target']=float(ps[0][1])
    

In [33]:
sub.head()

,image,target
0,xjngjjhzpyhxmstueplojbqreasnes.jpg,0.999748
1,mthrpqxgvwfdlksknvymngogpppnhp.jpg,0.000109
2,zgigmyufwdfgxuspzppusdfstysjfs.jpg,0.000005
3,hdmcngrvtgikfstjgrgqfotraeffgg.jpg,0.001499
4,rqxlpeljjhhifqnlrywertwvzywynb.jpg,0.990165


<h2>Dispaly images<h2>

In [0]:


show=sub
nrow = show.shape[0]
ncol = 1

fig = plt.figure(figsize=(4, 10)) 

gs = gridspec.GridSpec(nrow, ncol,
         wspace=0.0, hspace=0.0, top=0.95, bottom=0.05, left=0.17, right=0.845) 


for num, x in enumerate(show.image):
        img_path="/content/adem/images/"+str(x)
        im= PIL.Image.open(img_path).convert('RGB')
        ax= plt.subplot(gs[num,0])
        fig = plt.figure()
        fig.set_size_inches(150,150)
        ax.imshow(im)
        ax.set_xticklabels([])
        ax.set_yticklabels([])

plt.show()

<h2>Create CSV File<h2>

In [0]:
# we can round prediction to improve score to 0.009 ,and  without them score=0.015 
#sub.target=np.where(sub.target<0.18,0,sub.target)
#sub.target=np.where(sub.target>0.6,1,sub.target)
sub.to_csv('FinalSub.csv',index=False)
sub